In [ ]:
%%capture
!pip install llama-index-retrievers-bm25 llama-index==0.10.37 llama-index-embeddings-openai==0.1.9 qdrant-client==1.9.1 llama-index-vector-stores-qdrant==0.2.8 llama-index-llms-openai==0.1.19

In [ ]:
import os
import sys
from getpass import getpass
import nest_asyncio

from IPython.display import Markdown, display

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv("")

sys.path.append('../helpers')

from utils import setup_llm, setup_embed_model, setup_vector_store

In [ ]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or getpass("Enter your OpenAI API key: ")

In [ ]:
QDRANT_URL = ":memory:"

In [ ]:
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")

In [ ]:
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI
from utils import setup_llm, setup_embed_model

setup_llm(
    provider="openai", 
    model="gpt-4o", 
    api_key=OPENAI_API_KEY
    )

setup_embed_model(
    provider="openai", 
    model="text-embedding-3-small",
    api_key=OPENAI_API_KEY
    )

In [ ]:
import random
from utils import get_documents_from_docstore, group_documents_by_author, sample_documents

documents = get_documents_from_docstore("../data/words-of-the-senpais")

random.seed(42)

documents_by_author = group_documents_by_author(documents)

senpai_documents = sample_documents(documents_by_author, num_samples=5)

## Setup Qdrant Vector Store

In [ ]:
from llama_index.core import StorageContext
from llama_index.core.settings import Settings

from llama_index.core.node_parser import SentenceSplitter
from utils import create_index, create_query_engine, ingest, setup_vector_store

COLLECTION_NAME = "query-transforms"

vector_store = setup_vector_store(QDRANT_URL, QDRANT_API_KEY, COLLECTION_NAME)

sentence_splitter = SentenceSplitter(chunk_size=256, chunk_overlap=16)

nodes = ingest(
    documents=senpai_documents,
    transformations=[sentence_splitter, Settings.embed_model],
    vector_store=vector_store
)
index = create_index(
    from_where="vector_store",
    vector_store=vector_store,
    embed_model=Settings.embed_model,
    )

# Query Transformation

When handling user queries in a RAG system, agent, or any other pipeline, there are various ways to transform and decompose the queries before executing them.

One way is query rewriting. This involves rewriting the original query in multiple ways while which then sent sent for retrieval and generation. 

LlamaIndex implements various query transformations, [check the source code for details](https://github.com/run-llama/llama_index/blob/f116d75557d6867ed2cc61811a1c2f0b0c4d4ddb/llama-index-legacy/llama_index/legacy/indices/query/query_transform/base.py).


In [ ]:
from llama_index.core import PromptTemplate

QUERY_GEN_PROMPT = """Users aren't always the best at articulating what they're looking for. Your task is to understand the 
essense of the user query and generate {num_queries} alternate queries to expand the users query so it's more robust. This way the user will
recieve the most relevant information. 

Examples are delimited by triple backticks (```) below

````
User Query: How can I find the positive in situations that seem negative?

Alternate Queries:

1. How can I cultivate optimism and positive thinking in my daily life?
2. Is it possible to find meaning and purpose in challenging or difficult times?
3. What are some effective strategies for reframing negative thoughts into positive ones?
````

````
User Query: How do I deal with setbacks, failures, delays, defeat, or other disasters?

Alternate Queries:

1. How can I build resilience and learn to cope with adversity effectively?
2. What are some practical tips for overcoming challenges and obstacles that I face?
3. How can I develop a growth mindset and view setbacks as opportunities for learning?
4. What are healthy ways to process and learn from failures and mistakes?
````
````
User Query: How can I overcome defeat and suffering by changing my mindset?

Alternate Queries:

1. What is the power of positive thinking and affirmations, and how can they benefit me?
2. Can mindfulness and meditation practices improve my mental well-being and outlook?
3. How can I develop self-compassion and acceptance, especially during difficult times?
```

Generate {num_queries} alternate queries, one on each line, for the following user query:\n
--------------------
User Query: {query}\n
--------------------

Alternate Queries:\n
"""


QUERY_GEN_PROMPT_TEMPLATE = PromptTemplate(QUERY_GEN_PROMPT)

In [ ]:
QUERY_STRING = "How can I create my own luck?"

def generate_queries(query= QUERY_STRING, llm=Settings.llm, num_queries  = 4):
    response = llm.predict(
        QUERY_GEN_PROMPT_TEMPLATE, 
        num_queries=num_queries, 
        query=query
        )
    queries = response.split("\n")
    queries_str = "\n".join(queries)
    print(f"Generated queries:\n{queries_str}")
    return queries

generate_queries()

# `SubQuestionQueryEngine`

The `SubQuestionQueryEngine` works by breaking down a complex query into simpler sub-questions (with each potentially targeting a specific data source).

#### Here's how it works:

 - The `SubQuestionQueryEngine` receives a complex query.

- It then decomposes this query into several sub-questions. Each sub-question is designed to extract specific information from a particular data source.

- The engine then sends these sub-questions to their respective data sources and gathers the responses.

- Finally, it synthesizes all the intermediate responses to form a final comprehensive answer to the original complex query.

This process makes the `SubQuestionQueryEngine` particularly useful for handling compare/contrast queries across documents, as well as queries pertaining to a specific document. It's also well-suited for multi-document queries and can execute any number of sub-queries against any subset of query engine tools before synthesizing the final answer.

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

from prompts import HYPE_ANSWER_GEN_PROMPT

HYPE_ANSWER_GEN_PROMPT_TEMPLATE = PromptTemplate(HYPE_ANSWER_GEN_PROMPT)

query_engine_tools = [
    QueryEngineTool(
        query_engine=index.as_query_engine(),
        metadata=ToolMetadata(
            name="the senpais",
            description="The collective thoughts and writings of all my virtual mentors",
        ),
    ),
]

sub_question_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True
    )

sub_question_query_engine.update_prompts({'response_synthesizer:text_qa_template':HYPE_ANSWER_GEN_PROMPT_TEMPLATE})

In [ ]:
from utils import display_prompt_dict

sub_q_prompts = sub_question_query_engine.get_prompts()

display_prompt_dict(sub_q_prompts)

In [ ]:
sub_question_query_engine.query("How can I build my own luck, what are the types of luck I should pursue, and how can I hack luck and minimize my exposure to downside while maintaining skin in the game?")

# Hypothetical Document Embeddings (HyDE)

At a high level, [HyDE](https://arxiv.org/pdf/2212.10496.pdf) is an embedding technique that takes queries, generates a hypothetical answer, and then embeds that generated document and uses that as the final example. 

- 🧐 **Problem Tackled**: Addresses the struggle of creating fully zero-shot dense retrieval systems without relevance labels.

- 📚 **Traditional Methods**: Rely on relevance labels for document retrieval based on semantic similarities.

- 🚫 **Zero-Shot Challenge**: Especially tough without a large dataset for training.

### What is HyDE?

Given a query, `HyDE` instructs a language model to generate a hypothetical document.

This document captures relevance patterns but might contain inaccuracies or false details.

After generating the hypothetical document, an unsupervised contrastively learned encoder encodes the document into an embedding vector.

This vector identifies a neighborhood in the corpus embedding space, where similar real documents are retrieved based on vector similarity.

### How Does HyDE Work?

The process starts by feeding a query to a generative model with the instruction to "write a document that answers the question". This generates a hypothetical document that captures the essence of relevance.

 - Generates an embedding vector for a "fake" document

- It does not generate any actual text content for the document

- The embedding is solely for reserving space in the vectorstore index

- There is no full hypothetical document text you can access later

This vector is used to search against the corpus embeddings, and the most similar real documents are retrieved. The idea is that a hypothetical answer to a question is more semantically similar to the real answer than the question is. 

**In practice this means that your search would use GPT to generate a hypothetical answer, then embed that and use it for search**.

Key advantages of HyDE:

- Zero-shot, no labeled data or fine-tuning needed

- Performs comparably to fine-tuned retrievers across tasks/languages

- Grounds the query in real data via generated hypothetical documents

In [ ]:
from llama_index.core.indices.query.query_transform import HyDEQueryTransform

from llama_index.core.query_engine import TransformQueryEngine

In [ ]:
hyde = HyDEQueryTransform(
    include_original=True,
    )

hyde_query_engine = TransformQueryEngine(
    query_engine = index.as_query_engine(), 
    query_transform = hyde,
    )

In [ ]:
display_prompt_dict(hyde_query_engine.get_prompts())

In [ ]:
response = hyde_query_engine.query(QUERY_STRING)

display(Markdown(f"<b>{response}</b>"))

In [ ]:
from llama_index.core.query_pipeline import InputComponent
from utils import create_query_pipeline

input_component = InputComponent()

hyde_chain = [input_component,  hyde_query_engine]

hyde_query_pipeline = create_query_pipeline(hyde_chain)

In [ ]:
hyde_query_pipeline.run(input="What should I do if I feel like I am not being true to myself and relying too much on external securities?")